In [2]:
import sys
from Health_Afflictions import Poison, Healing
from Character_Classes import create_character
from Items import Weapon, Potion
from Save_Load_Game import save_game, load_game
from random import choice, randint

#Enemies
enemy_type = [
    ("Creepy Spooky Scary Skeleton", 40, 20, 10),
    ("The Giant Ogre", 80, 40, 20),
    ("Algathor the Dragon", 150, 60, 30)
]

character_classes = ["Mage", "Assassin", "Knight", "Archer"]
starting_item = [Weapon("Rusty Sword", 10).name, Potion("Health Potion", 20).name]

# Standalone Game Over function
def game_over():
    print("\n=== Game Over ===")
    return restart_game()

# Standalone Restart function
def restart_game():
    while True:
        restart = input("Would you like to restart? (Yes/No): ").lower().strip()
        if restart == "yes":
            print("\nRestarting game...\n")
            return True
        elif restart == "no":
            print("Goodbye!")
            return False
        else:
            print("Please enter 'Yes' or 'No' (type 'No' to exit)")

# Function to start the game
def start_game():
    print("Off to a land, filled with strife. All alone, but one with life. You were told to become a savior,")
    print("and yet no one will know your full story.")
    
    start_input = input("A story of One's Path of Legend! Type 'Start Game' to start!: ").strip()
    if start_input == "Start Game":
        print("You have chosen a treacherous path... Mother will be so proud of you...")
        return True  # Game can proceed
    else:
        print("WOW, there were some simple instructions to follow.")
        print("How the hell are you supposed to survive this narrative?")
        print("Screw you, here comes Bigfoot running towards you and slaps your head clean off.")
        return game_over()  # Return True for restart, False for exit

# Character creation function
def create_character():
    print("\nHello son, you here to make a Character Creation?")
    load = input("Load saved game? (Yes/No): ").lower().strip()
    if load == "yes":
        player = load_game()
        if player:
            print(f"Welcome back, {player.name}!")
            player.display_stats()
            return player
    name = input("Character Name: ").strip()
    print(f"Hello {name}, you sure are a brave one.")
    print("Choose your class:")
    print("Mage - Wields powerful magic, fragile")
    print("Assassin - Stealthy and quick")
    print("Knight - Strong and durable")
    print("Archer - Fires arrows with nice and clean shots")
    while True:
        class_type = input("Class: ").strip().capitalize()
        if class_type not in character_classes:
            print(f"Invalid class. Choose from {character_classes}")
            continue
        try:
            player = create_character(name, class_type)
            break
        except ValueError as e:
            print(e)
    print("Leaving the homestead and starting a journey. Grandpa didn't get there in time, running out of the shower naked.")
    print("You make your moves, comtemplating while becoming vigalant for every possiblility, at least you think.")
    for item in starting_item:
        player.inventory.append(item)
    player.display_stats()
    return player

#Level up function
def level_up(player):
    player.level += 1
    player.max_health += 10
    player.health = player.max_health
    player.strength += 5
    print(f"\n{player.name} has reached Level {player.level}!")
    print("Health and strength increased!")  

# Inventory management
def manage_inventory(player):
    print("\nOh lookie, your Inventory")
    if not player.inventory:
        print("Your inventory is empty.")
        return
    player.display_stats()
    action = input("Drop an item? (Yes/No): ").lower().strip()
    if action == "yes":
        print(f"Inventory: {player.inventory}")
        item = input("Which item to drop? ").strip()
        if item in player.inventory:
            player.inventory.remove(item)
            print(f"Dropped {item}.")
        else:
            print("Item not found!")

# Combat mechanics
def combat(player, enemy_name, enemy_health, enemy_strength, enemy_agility):
    enemy_hp = enemy_health
    print(f"\nBegin Your Combat Moves!")
    print(f"A {enemy_name} blocks your path!")
    while enemy_hp > 0 and player.health > 0:
        print(f"\n{enemy_name} HP: {enemy_hp}")
        player.display_stats()
        print("\nCombat Options:")
        print("1. Attack")
        print("2. Use Item")
        print("3. Save Game")
        print("4. Flee")
        action = input("Your choice: ").strip()
        
        if action == "1":
            damage = player.attack()
            enemy_hp -= damage
            if enemy_hp <= 0:
                print(f"\nYou vanquished the {enemy_name}!")
                if randint(1, 3) == 1:
                    level_up(player)
                break
        elif action == "2":
            if player.inventory:
                print(f"Inventory: {player.inventory}")
                item_name = input("Item to use: ").strip()
                if item_name in player.inventory:
                    if "Potion" in item_name:
                        potion = Potion(item_name, 20)
                        if potion.use(player):
                            player.inventory.remove(item_name)
                    elif "Sword" in item_name:
                        weapon = Weapon(item_name, 10 if "Rusty" in item_name else 15)
                        damage = weapon.damage + player.strength
                        enemy_hp -= damage
                        print(f"{player.name} uses {item_name}, dealing {damage} damage!")
                        if enemy_hp <= 0:
                            print(f"\nYou vanquished the {enemy_name}!")
                            if randint(1, 3) == 1:
                                level_up(player)
                            break
                    else:
                        print("Cannot use that item in combat!")
                else:
                    print("Item not found!")
            else:
                print("No items in inventory!")
        elif action == "3":
            save_game(player)
            print("Game saved. Combat continues...")
        elif action == "4":
            if randint(1, 100) <= player.agility:
                print(f"{player.name} flees successfully!")
                return True
            else:
                print("Failed to flee!")
        else:
            print("Invalid choice!")
            continue
        
        if enemy_hp > 0:
            dodge_chance = player.agility - enemy_agility
            if randint(1, 100) <= max(10, dodge_chance):
                print(f"{player.name} dodges the {enemy_name}'s attack!")
            else:
                damage = enemy_strength // 2
                player.health -= damage
                print(f"{enemy_name} strikes, dealing {damage} damage!")
                if player.health <= 0:
                    print(f"\nThe {enemy_name} has defeated you!")
                    return game_over()
    return True

# Exploration function
def explore(player):
    print("\nExploration of a lifetime")
    print("You tread deeper into the treacherous land...")
    event = randint(1, 4)
    if event == 1:
        item = choice([Weapon("Iron Sword", 15), Potion("Mana Potion", 20)])
        print(f"You discovered a {item.name}!")
        player.inventory.append(item.name)
    elif event == 2:
        enemy = choice(enemy_type)
        return combat(player, *enemy)
    elif event == 3:
        print("You find a moment of respite...")
        manage_inventory(player)
    else:
        print("The path is eerily silent...")

# Main game function
def play_game():
    should_restart = start_game()
    if should_restart is True:
        player = create_character()
        print("\nThe journey begins...")
        
        # Use Healing and Poison from Health_Afflictions
        healing = Healing(player.name, player.max_health, player.health)
        healing.heal_player(10)  # Example healing
        player.health = healing.health  # Sync health
        
        poison = Poison("Poison Cloud", 50, 50)
        poison.do_damage(10)  # Example poison effect
        
        encounters = 0
        max_encounters = 5
        while player.health > 0 and encounters < max_encounters:
            print(f"\n--- Encounter {encounters + 1}/{max_encounters} ---")
            explore(player)
            encounters += 1
            player.display_stats()
            print("\nWhat next?")
            print("1. Continue Journey")
            print("2. Manage Inventory")
            print("3. Save Game")
            print("4. Rest (End Game)")
            choice = input("Choice: ").strip()
            if choice == "1":
                continue
            elif choice == "2":
                manage_inventory(player)
            elif choice == "3":
                save_game(player)
            elif choice == "4":
                save_game(player)
                print(f"{player.name} rests, their legend paused...")
                return False
            else:
                print("Invalid choice, moving on...")
        
        if player.health > 0:
            print(f"Victory! {player.name} survived {max_encounters} encounters!")
            print("A true legend is born!")
            save_game(player)
        return False
    return should_restart


# Main game loop
running = True
while running:
    restart = play_game()
    if restart:
        continue
    else:
        running = False
print("Legend’s End")

TypeError: Items.__init__() takes 4 positional arguments but 5 were given